In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
dtype=torch.FloatTensor
sentences =["i like dog", "i love coffee", "i hate milk"]

word_list=" ".join(sentences).split()
word_list=list(set(word_list))
word_dict={w: i for i, w in enumerate(word_list)}
number_dict={i: w for i, w in enumerate(word_list)}
n_class=len(word_dict)

print("word_list: ", word_list)
print("word_dict: ", word_dict)
print("number_dict: ", number_dict)
print("n_class: ", n_class)

In [ ]:
n_step=2
n_hidden=2
# embedding_dim
m=2

In [ ]:
def make_batch(sentences):
    input_batch=[]
    target_batch=[]
    
    # Predict the last word of sentence    
    for sen in sentences:
        word=sen.split()
        input=[word_dict[n] for n in word[:-1]]
        target=word_dict[word[-1]]
        
        input_batch.append(input)
        target_batch.append(target)
        
    return input_batch, target_batch

In [ ]:
torch.unsqueeze(torch.tensor([3, 4, 5, 6, 7, 8, 9]), 0)

In [ ]:
# Embedding
embed_func=nn.Embedding(10, 4)
print(embed_func)
t1=torch.tensor([1, 2, 3, 4, 5]).view(1, -1)
print(t1, t1.size())
embed=embed_func(t1)
print(embed, embed.size())

embed_1=embed_func(torch.tensor([1, 2, 3, 4, 5]).view(1, -1))
print(embed_1, embed_1.size())
embed_2=embed_func(torch.tensor([3, 4, 5, 6, 7, 8, 9]).unsqueeze(0))
print(embed_2, embed_2.size())
embed_3=embed_func(torch.tensor([[3, 4, 5, 6, 7, 8, 9], [9, 7, 5, 3, 1, 2, 4]]))
print(embed_3, embed_3.size())

In [ ]:
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C=nn.Embedding(n_class, m) # num_embedding, embedding_dim
        self.H=nn.Parameter(torch.randn(n_step*m, n_hidden).type(dtype))
        self.W=nn.Parameter(torch.randn(n_step*m, n_class).type(dtype))
        self.d=nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.U=nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))
        self.b=nn.Parameter(torch.randn(n_class).type(dtype))
    
    def forward(self, X):
        X=self.C(X) # batch_size, n_step, embedding_dim
        X=X.view(-1, n_step*m) # batch_size, (n_step*embedding_dim)
        # (batch_size, n_hidden)=n_hidden + torch.mm((batch_size, (n_step*embedding_dim)), (n_step*embedding_dim, n_hidden))
        tanh=torch.tanh(self.d+torch.mm(X, self.H)) 
        # (batch_size, n_class)=mm((batch_size, (n_step*embedding_dim)), (n_step*embedding_dim, n_class))+mm((batch_size, n_hidden), (n_hidden, n_class))
        output=self.b+torch.mm(X, self.W)+torch.mm(tanh, self.U)
        return output

In [ ]:
model=NNLM()

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

input_batch, target_batch=make_batch(sentences)
input_batch=torch.tensor(torch.LongTensor(input_batch))
target_batch=torch.tensor(torch.LongTensor(target_batch))

In [ ]:
list(model.parameters())

In [ ]:
for epoch in range(5000):
    optimizer.zero_grad()
    output=model(input_batch)
    
    loss=criterion(output, target_batch)
    if (epoch+1)%1000==0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        
    loss.backward()
    optimizer.step()
    
predict=model(input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])